In [41]:
import librosa as librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import librosa
from sklearn.svm import SVC 
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix


In [42]:

header = 'filename rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

for i in range(1, 40):
    header += f' mfcc{i}'
header = header.split()
# print('CSV Header: ', header)
file = open("data.csv", 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()


def writeCsv(data):
    file = open("data.csv", 'a', newline='')
    writer = csv.writer(file)
    writer.writerow(data.split(","))
    file.close()



def feature_extraction(file_path):
    # iterate through all file
        y, sr = librosa.load(file_path, mono=True, duration=30)
            # remove leading and trailing silence
        y, index = librosa.effects.trim(y)
        # chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=39)
        to_append = f'{file},{np.mean(rmse)},{np.mean(spec_cent)},{np.mean(spec_bw)},{np.mean(rolloff)},{np.mean(zcr)}'
        for e in mfcc:
            to_append += f',{np.mean(e)}'
        
        writeCsv(to_append)
        return to_append

to_append =[]





path ='./Wav/Train_2/'

for file in os.listdir(path):
    # print(file)
    file_path = f"{path}\{file}"  
    data = feature_extraction(file_path)








    

In [43]:
def feature_extraction_array(file_path):
    to_append =[]
    # iterate through all file
    y, sr = librosa.load(file_path, mono=True, duration=30)
        # remove leading and trailing silence
    y, index = librosa.effects.trim(y)
    # chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=39)
    to_append.append(np.mean(rmse))
    to_append.append(np.mean(spec_cent))
    to_append.append(np.mean(spec_bw))
    to_append.append(np.mean(rolloff))
    to_append.append(np.mean(zcr))
    for e in mfcc:
        to_append.append(np.mean(e))
    return to_append
    
    
def preProcessing(csvName):
    data = pd.read_csv(csvName)
    audioName = data['filename']
    speakerNumber = []
    for i in range(len(audioName)):
        speakerLetter = audioName[i][0]
        #Shrouk----2
        #Rawda---0
        #Habiba---1
        if speakerLetter == "A":
            speakerLetter = "0"
        elif speakerLetter =="H":
            speakerLetter="1"
        elif speakerLetter == "M":
            speakerLetter = "2"
        elif speakerLetter =="Y":
            speakerLetter = "3"
        else:
            speakerLetter = "4"
        speakerNumber.append(speakerLetter)

    data = data.drop(['filename'],axis=1)


    print(data.tail())
    
    return data, speakerNumber


***Speaker Recognition***

In [44]:
data, number = preProcessing("data.csv")


        rmse  spectral_centroid  spectral_bandwidth      rolloff  \
37  0.013293        2649.866632         2092.284144  5040.411894   
38  0.012756        2573.992811         2041.769303  4744.715465   
39  0.010821        2414.947596         1948.162513  4413.985250   
40  0.010712        2473.364151         1999.682209  4546.398379   
41  0.013799        2216.317597         1815.950663  4019.774563   

    zero_crossing_rate       mfcc1       mfcc2     mfcc3      mfcc4  \
37            0.193575 -512.373047   82.252724 -5.832910  20.100780   
38            0.199942 -530.374573   81.688698  4.311122  13.437059   
39            0.186392 -525.386292   84.499336  2.854430  17.898985   
40            0.189912 -533.232849   88.442421  4.224247  22.496590   
41            0.160562 -510.668365  101.569267  5.563087  21.911745   

        mfcc5  ...    mfcc30    mfcc31    mfcc32    mfcc33    mfcc34  \
37 -14.369832  ... -2.343000 -4.666240 -3.466050 -1.153331 -4.116298   
38 -14.103704  ... -

In [45]:
X= np.array(data)

y=np.asarray(number)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=123
    )


In [47]:
svm_model= svm.SVC(kernel='linear' , C=0.001)
svm_model.fit(X_train, y_train) 

y_pred = svm_model.predict(X_test)



In [8]:
testing = feature_extraction_array('Sara-11.wav')
y_predict = svm_model.predict([testing])



print(y_predict)


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'Sara-11.wav'

In [48]:
from sklearn import metrics
y_pred_t = svm_model.predict(X_train)
accuracy_t = metrics.accuracy_score(y_train,y_pred_t)


print(f"  Train model accuracy = {100* accuracy_t:.3} %")
accuracy = metrics.accuracy_score(y_test,y_pred)


print(f"  Test model accuracy = {100* accuracy:.3} %")


  Train model accuracy = 1e+02 %
  Test model accuracy = 88.9 %


***Spoken Digit Recognition***